<a href="https://colab.research.google.com/github/flecue/GAN-AniMe/blob/main/Anime_Facegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit: https://github.com/bryandlee/animegan2-pytorch

Interactive Demo for AnimeGANv2:FacePortraitv2 created by @xhlulu

Learn more about the model here: https://github.com/bryandlee/animegan2-pytorch

To start using this, run the cells with `Ctrl+F9` or "Runtime > Run All"

For accelerated inference, you can use a GPU. Simply select "Runtime > Change runtime type" and select "GPU" in the "Hardware Acceleration" dropdown.


In [ ]:
#@title Anime FaceGAN Colab app

from io import BytesIO
import torch
from PIL import Image

import ipywidgets as widgets
import IPython.display as display
from google.colab import files

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)
image_format = "png" #@param ["jpeg", "png"]

button = widgets.Button(description="Start")
output = widgets.Output()


def run(b):
    button.disabled = True

    with output:
        display.clear_output()
    
    uploaded = files.upload()

    for fname in uploaded:
        bytes_in = uploaded[fname]

        im_in = Image.open(BytesIO(bytes_in)).convert("RGB")
        im_out = face2paint(model, im_in, side_by_side=False)
        buffer_out = BytesIO()
        im_out.save(buffer_out, format=image_format)
        bytes_out = buffer_out.getvalue()
        wi1 = widgets.Image(value=bytes_in, format=image_format)
        wi2 = widgets.Image(value=bytes_out, format=image_format)
        wi1.layout.max_width = '500px'
        wi1.layout.max_height = '500px'
        wi2.layout.max_width = '500px'
        wi2.layout.max_height = '500px'

        ## Side by side thanks to HBox widgets
        sidebyside = widgets.HBox([wi1, wi2])
        ## Finally, show.
        with output:
            display.display(sidebyside)
    
    button.disabled = False

button.on_click(run)
display.display(button, output)